In [1]:
import torch,torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422
...,...,...,...,...,...
554,vid_4_9860.jpg,0.000000,198.321729,49.235251,236.223284
555,vid_4_9880.jpg,329.876184,156.482351,536.664239,250.497895
556,vid_4_9900.jpg,0.000000,168.295823,141.797524,239.176652
557,vid_4_9960.jpg,487.428988,172.233646,616.917699,228.839864


In [4]:
pred = {'instances':{'pred_classes':[],'pred_boxes':[]}}

In [5]:
# pred['instances'].pred_classes

In [6]:
xmin,ymin,xmax,ymax = 281,187,327,223

In [7]:
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin

In [8]:
im = cv2.imread('./data/vid_4_1000.jpg')
roi=im[y:y+h,x:x+w]
cv2.imwrite(str('crop') + '.jpg', roi)
cv2.rectangle(im,(x,y),(x+w,y+h),(200,0,0),2)
cv2.imwrite(str('box') + '.jpg', im)

True

In [9]:
data

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422
...,...,...,...,...,...
554,vid_4_9860.jpg,0.000000,198.321729,49.235251,236.223284
555,vid_4_9880.jpg,329.876184,156.482351,536.664239,250.497895
556,vid_4_9900.jpg,0.000000,168.295823,141.797524,239.176652
557,vid_4_9960.jpg,487.428988,172.233646,616.917699,228.839864


In [10]:
def load_data():
    new_data = []
    idx = len(data)
    for i in tqdm(range(idx)):
        record = {}
        info = data.iloc[i]
        img_path = f'./data/{info["image"]}'
        img = cv2.imread(f'./data/{info["image"]}')
        img = img / 255.0
        record['file_name'] = img_path
        record['image_id'] = i
        record['height'],record['width'] = img.shape[:2]
        objs = [{
            'bbox':[xmin,ymin,xmax,ymax],
            'bbox_mode':BoxMode.XYXY_ABS,
            'category_id':0
        }]
        record['annotations'] = objs
        new_data.append(record)
    return new_data

In [11]:
labels = ['car']

In [12]:
DatasetCatalog.register('data',lambda : load_data())
MetadataCatalog.get('data').set(thing_classes=labels)
metadata = MetadataCatalog.get('data')

In [13]:
wandb.init(sync_tensorboard=True,name='baseline')
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_C4_3x.yaml'))
cfg.DATASETS.TRAIN = ('data',)
cfg.DATASETS.TEST = ()
cfg.TEST.EVAL_PERIOD = 100
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_101_C4_3x.yaml')
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 2500
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
predictor = DefaultPredictor(cfg)

2021-10-11 15:46:15.700100: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.0
wandb: Run data is saved locally in wandb/run-20211011_154617-1r73t36y
wandb: Syncing run baseline



[10/11 15:46:33 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)


100%|████████████████████████████████████████| 559/559 [00:02<00:00, 233.44it/s]


[10/11 15:46:35 d2.data.build]: Removed 0 images with no usable annotations. 559 images left.
[10/11 15:46:35 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|    car     | 559          |
|            |              |
[10/11 15:46:35 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[10/11 15:46:35 d2.data.build]: Using training sampler TrainingSampler
[10/11 15:46:35 d2.data.common]: Serializing 559 elements to byte tensors and concatenating them all ...
[10/11 15:46:35 d2.data.common]: Serialized dataset takes 0.19 MiB


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 2048) in the checkpoint but (2, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 2048) in the checkpoint but (4, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[10/11 15:46:36 d2.engine.train_loop]: Starting training from iteration 0
[10/11 15:46:48 d2.utils.events]:  eta: 0:23:34  iter: 19  total_loss: 0.8574  loss_cls: 0.7535  loss_box_reg: 0.03398  loss_rpn_cls: 0.04717  loss_rpn_loc: 0.003807  time: 0.5675  data_time: 0.0140  lr: 4.9953e-06  max_mem: 3156M
[10/11 15:46:59 d2.utils.events]:  eta: 0:23:00  iter: 39  total_loss: 0.7439  loss_cls: 0.6396  loss_box_reg: 0.03539  loss_rpn_cls: 0.04951  loss_rpn_loc: 0.003864  time: 0.5626  data_time: 0.0033  lr: 9.9902e-06  max_mem: 3156M
[10/11 15:47:10 d2.utils.events]:  eta: 0:22:41  iter: 59  total_loss: 0.5374  loss_cls: 0.4656  loss_box_reg: 0.000559  loss_rpn_cls: 0.05131  loss_rpn_loc: 0.003837  time: 0.5565  data_time: 0.0030  lr: 1.4985e-05  max_mem: 3156M
[10/11 15:47:21 d2.utils.events]:  eta: 0:22:29  iter: 79  total_loss: 0.3946  loss_cls: 0.303  loss_box_reg: 0.02881  loss_rpn_cls: 0.04633  loss_rpn_loc: 0.002917  time: 0.5560  data_time: 0.0028  lr: 1.998e-05  max_mem: 3156M
[10

In [14]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [15]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [16]:
import matplotlib.pyplot as plt

In [17]:
from detectron2.utils.visualizer import ColorMode
im = cv2.imread('./data/vid_4_1000.jpg')
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=metadata, 
               scale=0.5, 
               instance_mode=ColorMode.IMAGE_BW
              )
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize=(12,6))
plt.imshow(out.get_image()[:, :, ::-1])
plt.savefig('./pred.png')
plt.close()

In [18]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "data")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[10/11 16:07:33 d2.evaluation.coco_evaluation]: 'data' is not registered by `register_coco_instances`. Therefore trying to convert it to COCO format ...
WARNING [10/11 16:07:33 d2.data.datasets.coco]: Using previously cached COCO format annotations at './output/data_coco_format.json'. You need to clear the cache file if your dataset has been modified.


100%|████████████████████████████████████████| 559/559 [00:02<00:00, 259.43it/s]

[10/11 16:07:35 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[10/11 16:07:35 d2.data.common]: Serializing 559 elements to byte tensors and concatenating them all ...


[10/11 16:07:35 d2.data.common]: Serialized dataset takes 0.19 MiB
[10/11 16:07:35 d2.evaluation.evaluator]: Start inference on 559 batches
[10/11 16:07:38 d2.evaluation.evaluator]: Inference done 11/559. Dataloading: 0.0009 s/iter. Inference: 0.2805 s/iter. Eval: 0.0001 s/iter. Total: 0.2816 s/iter. ETA=0:02:34
[10/11 16:07:43 d2.evaluation.evaluator]: Inference done 30/559. Dataloading: 0.0010 s/iter. Inference: 0.2740 s/iter. Eval: 0.0001 s/iter. Total: 0.2752 s/iter. ETA=0:02:25
[10/11 16:07:49 d2.evaluation.evaluator]: Inference done 50/559. Dataloading: 0.0010 s/iter. Inference: 0.2683 s/iter. Eval: 0.0001 s/iter. Total: 0.2695 s/iter. ETA=0:02:17
[10/11 16:07:54 d2.evaluation.evaluator]: Inference done 69/559. Dataloading: 0.0010 s/iter. Inference: 0.2677 s/iter. Eval: 0.0001 s/iter. Total: 0.2689 s/iter. ETA=0:02:11
[10/11 16:07:59 d2.evaluation.evaluator]: Inference done 89/559. Dataloading: 0.0010 s/iter. Inference: 0.2653 s/iter. Eval: 0.0001 s/iter. Total: 0.2665 s/iter. ET

In [19]:
wandb.log({'coco':inference_on_dataset(predictor.model, val_loader, evaluator)})

[10/11 16:10:02 d2.evaluation.evaluator]: Start inference on 559 batches
[10/11 16:10:05 d2.evaluation.evaluator]: Inference done 11/559. Dataloading: 0.0008 s/iter. Inference: 0.2593 s/iter. Eval: 0.0001 s/iter. Total: 0.2603 s/iter. ETA=0:02:22
[10/11 16:10:10 d2.evaluation.evaluator]: Inference done 31/559. Dataloading: 0.0009 s/iter. Inference: 0.2581 s/iter. Eval: 0.0001 s/iter. Total: 0.2593 s/iter. ETA=0:02:16
[10/11 16:10:15 d2.evaluation.evaluator]: Inference done 51/559. Dataloading: 0.0009 s/iter. Inference: 0.2582 s/iter. Eval: 0.0001 s/iter. Total: 0.2593 s/iter. ETA=0:02:11
[10/11 16:10:20 d2.evaluation.evaluator]: Inference done 71/559. Dataloading: 0.0009 s/iter. Inference: 0.2580 s/iter. Eval: 0.0001 s/iter. Total: 0.2591 s/iter. ETA=0:02:06
[10/11 16:10:25 d2.evaluation.evaluator]: Inference done 91/559. Dataloading: 0.0009 s/iter. Inference: 0.2575 s/iter. Eval: 0.0001 s/iter. Total: 0.2587 s/iter. ETA=0:02:01
[10/11 16:10:31 d2.evaluation.evaluator]: Inference done 

In [20]:
wandb.log(inference_on_dataset(predictor.model, val_loader, evaluator))

[10/11 16:12:27 d2.evaluation.evaluator]: Start inference on 559 batches
[10/11 16:12:30 d2.evaluation.evaluator]: Inference done 11/559. Dataloading: 0.0008 s/iter. Inference: 0.2559 s/iter. Eval: 0.0001 s/iter. Total: 0.2568 s/iter. ETA=0:02:20
[10/11 16:12:35 d2.evaluation.evaluator]: Inference done 31/559. Dataloading: 0.0010 s/iter. Inference: 0.2585 s/iter. Eval: 0.0001 s/iter. Total: 0.2597 s/iter. ETA=0:02:17
[10/11 16:12:40 d2.evaluation.evaluator]: Inference done 51/559. Dataloading: 0.0010 s/iter. Inference: 0.2580 s/iter. Eval: 0.0001 s/iter. Total: 0.2592 s/iter. ETA=0:02:11
[10/11 16:12:45 d2.evaluation.evaluator]: Inference done 71/559. Dataloading: 0.0010 s/iter. Inference: 0.2568 s/iter. Eval: 0.0001 s/iter. Total: 0.2580 s/iter. ETA=0:02:05
[10/11 16:12:50 d2.evaluation.evaluator]: Inference done 90/559. Dataloading: 0.0010 s/iter. Inference: 0.2582 s/iter. Eval: 0.0001 s/iter. Total: 0.2594 s/iter. ETA=0:02:01
[10/11 16:12:55 d2.evaluation.evaluator]: Inference done 